<a href="https://colab.research.google.com/github/JarupongOfficial/maskDetection/blob/main/trainModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Initial Google Collab to Github")

Initial Google Collab to Github
